# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [64]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [65]:
# 1) Seu código aqui
URL = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
req = requests.get(URL)
print(req.status_code)
data_json = req.json()
print(data_json['items'])

200
[{'id_pleito': 9050, 'tipo_interessado': 'Estado', 'interessado': 'Rio de Janeiro', 'cod_ibge': 33, 'uf': 'RJ', 'num_pvl': None, 'status': 'Encaminhado à PGFN com manifestação técnica favorável', 'num_processo': '17944.000893/2010-16', 'data_protocolo': '2011-06-03T03:00:00Z', 'tipo_operacao': 'Operação contratual interna (com garantia da União)', 'finalidade': 'Infraestrutura', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Caixa Econômica Federal', 'moeda': 'Real', 'valor': 259138332.31, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '04/07/2011'}, {'id_pleito': 6660, 'tipo_interessado': 'Estado', 'interessado': 'Rio de Janeiro', 'cod_ibge': 33, 'uf': 'RJ', 'num_pvl': None, 'status': 'Deferido', 'num_processo': '17944.000403/2010-81', 'data_protocolo': '2010-05-24T03:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'PAC - Favelas', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Caixa Econômica Federal', 'moeda':

In [66]:
# 2) Seu código aqui
df = pd.DataFrame(data_json['items'])
status_disponivel = df['status'].value_counts()
print('#3 Status mais Solicitados#')
print(status_disponivel[0:3])


#3 Status mais Solicitados#
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: status, dtype: int64


In [67]:
# 3) Seu código aqui
ano_status=[]
i=0
while i < len(df) :
    ano_status.append(df['data_status'][i].split(sep='/')[2])
    i=i+1

df['ano_status_df']=ano_status
freq = df['ano_status_df'].value_counts()
print('Frequência por Ano')
print(freq)


Frequência por Ano
2012    11
2009     9
2013     7
2011     6
2016     4
2002     4
2014     4
2010     3
2008     2
2006     2
Name: ano_status_df, dtype: int64


2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [68]:
#1) Seu código aqui
def consulta (uf,interessado):
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=' +uf+'&tipo_interessado='+interessado
    req_1 = requests.get(url)
    data_json_1 = req_1.json()
    df_1 = pd.DataFrame(data_json_1['items'])
    return df_1
df_consultado = consulta('RJ','Estado')

print(df_consultado)
 

    id_pleito tipo_interessado     interessado  cod_ibge  uf  \
0        9050           Estado  Rio de Janeiro        33  RJ   
1        6660           Estado  Rio de Janeiro        33  RJ   
2        9898           Estado  Rio de Janeiro        33  RJ   
3       22594           Estado  Rio de Janeiro        33  RJ   
4       10606           Estado  Rio de Janeiro        33  RJ   
5       12410           Estado  Rio de Janeiro        33  RJ   
6        5918           Estado  Rio de Janeiro        33  RJ   
7        6833           Estado  Rio de Janeiro        33  RJ   
8       13271           Estado  Rio de Janeiro        33  RJ   
9        7080           Estado  Rio de Janeiro        33  RJ   
10       9750           Estado  Rio de Janeiro        33  RJ   
11       8556           Estado  Rio de Janeiro        33  RJ   
12      10002           Estado  Rio de Janeiro        33  RJ   
13       5933           Estado  Rio de Janeiro        33  RJ   
14       5952           Estado  Rio de J

In [70]:
# 2) Seu código aqui
df_consultado = consulta('MG','Estado')
df_consultado_status = df_consultado[df_consultado['status']=='Arquivado por decurso de prazo']
print('Solicitações Arquivadas por decurso de prazo')
print(df_consultado_status['status'].value_counts())


Solicitações Arquivadas por decurso de prazo
Arquivado por decurso de prazo    1
Name: status, dtype: int64


In [71]:
# 3) Seu código aqui
df_consultado = consulta('BA','Município')
df_consultado_deferido = df_consultado[df_consultado['status']=='Deferido']
#print(len(df_consultado))
#print(len(df_consultado_deferido))
#print(df_consultado.head(5))
print('Município da Bahia com mais solicitações deferidas')
print(df_consultado_deferido['interessado'].value_counts()[0:1])

Município da Bahia com mais solicitações deferidas
Luís Eduardo Magalhães    14
Name: interessado, dtype: int64


In [72]:
# 4) Seu código aqui
df_consultado = consulta('BA','Estado')
df_consultado.to_csv('BA_Estado.csv', index=False, sep=';', decimal=',')